In [ ]:
# Author: Naveen Lalwani
# Script to distill knowledge of LeNet-300-100 model trained on MNIST to student model

import tensorflow as tf
import numpy as np
import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.models import Model
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import RMSprop, SGD, Adam

In [ ]:
# Preprocessing for smaller model
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# Normalization
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# One Hot Encoding
y_train = to_categorical(y_train.astype('float32'))
y_test = to_categorical(y_test.astype('float32'))

In [ ]:
# Teacher Model: LeNet-300-100
def lenet_300_100_model():
    inputs = layers.Input(shape = (784,))
    
    x = layers.Dense(300, activation='relu', name='FC1')(inputs)
    
    x = layers.Dense(100, activation='relu', name='FC2')(x)

    x = layers.Dense(10, name='logits')(x)
    preds = layers.Activation('softmax', name='Softmax')(x)

    model = Model(inputs=inputs, outputs=preds)
    model.summary()
    return model

#**Build Model LeNet-300-100**

In [62]:
model = lenet_300_100_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 784)               0         
_________________________________________________________________
FC1 (Dense)                  (None, 300)               235500    
_________________________________________________________________
FC2 (Dense)                  (None, 100)               30100     
_________________________________________________________________
logits (Dense)               (None, 10)                1010      
_________________________________________________________________
Softmax (Activation)         (None, 10)                0         
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [63]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(x_train, y_train, epochs=5, batch_size = 512) 

Epoch 1/5
60000/60000 [==============================] - 3s 44us/sample - loss: 0.4256 - categorical_accuracy: 0.8852
Epoch 2/5
60000/60000 [==============================] - 2s 40us/sample - loss: 0.1568 - categorical_accuracy: 0.9542
Epoch 3/5
60000/60000 [==============================] - 2s 40us/sample - loss: 0.1070 - categorical_accuracy: 0.9691
Epoch 4/5
60000/60000 [==============================] - 2s 40us/sample - loss: 0.0783 - categorical_accuracy: 0.9774
Epoch 5/5
60000/60000 [==============================] - 2s 40us/sample - loss: 0.0607 - categorical_accuracy: 0.9825


In [64]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

10000/10000 [==============================] - 1s 76us/sample - loss: 0.0811 - categorical_accuracy: 0.9759
Test Loss: 0.08111305869612843
Test Accuracy: 0.9759


In [ ]:
getSoftmaxKnowledge = Model(inputs=model.input, outputs=model.get_layer("logits").output)
model_logits = getSoftmaxKnowledge.predict(x_train)

In [ ]:
# Defining function described by Geoffrey Hinton in his paper of Knowledge Distillation
def softmax_with_temperature(logits, temperature):
    logits = logits / temperature
    return (np.exp(logits) / np.sum(np.exp(logits)))

In [ ]:
# Temperature is a hyperparameter
temperature = 3
softened_train_prob = softmax_with_temperature(model_logits, temperature)

In [ ]:
# Model Definition for the Student Model
def build_small_model():
    inputs = layers.Input(shape = (784,))
    
    x = layers.Dense(50, activation='relu', name='FC1')(inputs)
    
    x = layers.Dense(10, name='logits')(x)
    
    preds = layers.Activation('softmax', name='Softmax')(x)
  
    model = Model(inputs=inputs, outputs=preds)
    model.summary()
    return model

In [123]:
small_model = build_small_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 784)               0         
_________________________________________________________________
FC1 (Dense)                  (None, 50)                39250     
_________________________________________________________________
logits (Dense)               (None, 10)                510       
_________________________________________________________________
Softmax (Activation)         (None, 10)                0         
Total params: 39,760
Trainable params: 39,760
Non-trainable params: 0
_________________________________________________________________


# **Distilling Knowledge in the student model**

In [125]:
# Optimization = Adam
# Loss = Cross Entropy loss
# Epochs = 50
# Trained with dark knowledge

small_model.compile(optimizer='adam', loss= 'categorical_crossentropy', metrics=['categorical_accuracy'])
small_model.fit(x_train, softened_train_prob, epochs=50, batch_size=128)

test_loss, test_acc = small_model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

Epoch 1/50
60000/60000 [==============================] - 2s 36us/sample - loss: 8.9660e-06 - categorical_accuracy: 0.9817
Epoch 2/50
60000/60000 [==============================] - 2s 27us/sample - loss: 8.9663e-06 - categorical_accuracy: 0.9816
Epoch 3/50
60000/60000 [==============================] - 2s 28us/sample - loss: 8.9658e-06 - categorical_accuracy: 0.9818
Epoch 4/50
60000/60000 [==============================] - 2s 28us/sample - loss: 8.9649e-06 - categorical_accuracy: 0.9818
Epoch 5/50
60000/60000 [==============================] - 2s 29us/sample - loss: 8.9644e-06 - categorical_accuracy: 0.9818
Epoch 6/50
60000/60000 [==============================] - 2s 28us/sample - loss: 8.9636e-06 - categorical_accuracy: 0.9819
Epoch 7/50
60000/60000 [==============================] - 2s 29us/sample - loss: 8.9626e-06 - categorical_accuracy: 0.9820
Epoch 8/50
60000/60000 [==============================] - 2s 28us/sample - loss: 8.9619e-06 - categorical_accuracy: 0.9824
Epoch 9/50
60000

In [ ]:
small_model.save('model_50_LeNet-300-100_Distilled.h5')
model.save('model_LeNet-300-100.h5')